In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('whitegrid')

In [2]:
df_test_features = pd.read_csv('../test_features.csv')

In [3]:
num_test_data = df_test_features.iloc[:,9:]

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [5]:
num_test = scaler.fit_transform(num_test_data)

In [6]:
import ktrain

In [7]:
predictor = ktrain.load_predictor('../audio_text_nbsvm_model')

# Pre-process string with ktrain

In [8]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [9]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [10]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [11]:
x_text_input = predictor.preproc.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


# Make a prediction

In [12]:
test_dataset = [num_test] + [x_text_input[0]]

In [13]:
prediction = predictor.model.predict(test_dataset)

In [14]:
predict_labels = [ label.argmax() for label in prediction]

In [15]:
df = pd.DataFrame(predict_labels, columns=['genre_pred'])

In [16]:
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}

In [17]:
df['genre_pred'] = df['genre_pred'].map(classes_mapping)

In [18]:
df['trackID'] = df_test_features['trackID']

In [19]:
correct_label = pd.read_csv('../test_labels.csv')

In [20]:
# merge df and correct_label by trackID
df_merged = pd.merge(df, correct_label, on='trackID')

In [21]:
from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.30      0.33      0.32        64
dance and electronica       0.48      0.19      0.27        64
                 folk       0.51      0.64      0.57        73
       jazz and blues       0.12      0.06      0.08        36
                metal       0.76      0.60      0.67        42
                  pop       0.56      0.98      0.71        44
                 punk       0.41      0.50      0.45        40
      soul and reggae       0.41      0.42      0.41        65

             accuracy                           0.46       428
            macro avg       0.44      0.46      0.43       428
         weighted avg       0.45      0.46      0.43       428

